# Viterbi in Log Space

This notebook contains code to convert the Viterbi algorithm to
log space, which will fix the problem with underflow creating
the `Key Error: None` that some of you have seen. 

Another feature of this patch is that it avoids the creation
of arrays for the statistics on starting, transition, and 
emission probabilities. 

In [3]:
import math
import numpy as np
from numpy.random import shuffle, seed, choice
from tqdm import tqdm
from collections import defaultdict


In [4]:
# Calculate statistics from part of tagged sentences

# First, create dictionary of starting frequencies for each tag

start_p = defaultdict(int)

# etc. to create start_p
    
# now convert it into log probabilities
    
start_p_log = defaultdict(lambda: float('-inf'))

for tag in start_p:
    start_p_log[tag] = np.log( start_p[tag] )
                
start_p_log

defaultdict(<function __main__.<lambda>()>, {})

In [5]:
# First, create nested dictionary of transition frequencies for each tag
#  where t2 follows t1

trans_p = defaultdict(lambda: defaultdict(int))

# etc to create trans_p

# now convert it into log probabilities
        
trans_p_log = defaultdict(lambda: defaultdict(lambda: float('-inf')))

for t1 in trans_p:
    for t2 in trans_p[t1]:
        trans_p_log[t1][t2] = np.log( trans_p[t1][t2] )
        

In [6]:
# First, create nested dictionary of emission frequencies for each tag:
# given tag, how many times does word occur with that tag?

emit_p = defaultdict(lambda: defaultdict(int))

# etc to create emit_p
    
# now convert it into log probabilities
        
emit_p_log = defaultdict(lambda: defaultdict(lambda: float('-inf')))

for t in emit_p:
    for w in emit_p[t]:
        emit_p_log[t][w] = np.log( emit_p[t][w] )



In [7]:
import numpy as np

# Adapted from the Wikipedia article just referenced
# Obs_sequence is sequence of words "observed"; returns
# most likely sequence of states = POS. 

# start_p, trans_p, emit_p are dictionaries giving probabilities
# set logspace to False if these are normal probabilities (not in log space)

def viterbi(obs_sequence, obs, states, start_p, trans_p, emit_p,logspace=True):
    
    V = [{}]
    for st in states:
        if logspace:
            V[0] [st] = {"prob": start_p[st] + emit_p[st][obs_sequence[0]], "prev": None}
        else:
            V[0] [st] = {"prob": start_p[st] * emit_p[st][obs_sequence[0]], "prev": None}
        
    # Run Viterbi when t > 0
    
    for t in range(1, len(obs_sequence)):
        V.append({})
        for st in states:
            
            if logspace:
                max_tr_prob = V[t - 1] [states[0]] ["prob"] + trans_p[states[0]] [st]
            else:
                max_tr_prob = V[t - 1] [states[0]] ["prob"] * trans_p[states[0]] [st]
                
            prev_st_selected = states[0]
            
            for prev_st in states[1:]:
                
                if logspace:
                    tr_prob = V[t - 1] [prev_st] ["prob"] + trans_p[prev_st] [st]
                else:
                    tr_prob = V[t - 1] [prev_st] ["prob"] * trans_p[prev_st] [st]
                    
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            if logspace:
                max_prob = max_tr_prob + emit_p[st] [obs_sequence[t]]
            else:
                max_prob = max_tr_prob * emit_p[st] [obs_sequence[t]]
                
            V[t] [st] = {"prob": max_prob, "prev": prev_st_selected}

    opt = []
    max_prob = float('-inf')
    best_st = None

    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1] [previous] ["prev"])
        previous = V[t + 1] [previous] ["prev"]

    return (opt,max_prob)

In [8]:
# Try it with normal probabilities

sent = tagged_sentences[3]
(s,t) = list(zip(*sent))
(t_hat,p) = viterbi(s, np.array(all_words), np.array(all_tags),start_p,trans_p,emit_p,logspace=False)

for k in range(len(s)):
    print(f'{s[k]:20}\t{t[k]:10}\t{t_hat[k]:10}')

NameError: name 'tagged_sentences' is not defined

In [14]:
# try it with log probabilities

sent = tagged_sentences[3]
(s,t) = list(zip(*sent))
(t_hat,p) = viterbi(s, np.array(all_words), np.array(all_tags),start_p_log,trans_p_log,emit_p_log)

for k in range(len(s)):
    print(f'{s[k]:20}\t{t[k]:10}\t{t_hat[k]:10}')

She                 	PRON      	PRON      
said                	VERB      	VERB      
,                   	.         	.         
``                  	.         	.         
I                   	PRON      	PRON      
guess               	VERB      	VERB      
the                 	DET       	DET       
Lord                	NOUN      	NOUN      
looks               	VERB      	VERB      
out                 	PRT       	PRT       
for                 	ADP       	ADP       
fools               	NOUN      	NOUN      
,                   	.         	.         
drunkards           	NOUN      	NOUN      
,                   	.         	.         
and                 	CONJ      	CONJ      
innocents           	NOUN      	NOUN      
''                  	.         	.         
.                   	.         	.         
